## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50

In [2]:
batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練整個資料集共 30個循環

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train shape (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
x_train = x_train / 255
x_test = x_test / 255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [5]:
# transfer learning
resnet_model = ResNet50(input_shape=(32,32,3),
                        weights='imagenet', 
                        pooling='avg',
                        include_top=False)

last_featuremaps = resnet_model.output
x = Flatten()(last_featuremaps)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax', name='output')(x)

New_resnet_model = Model(inputs=resnet_model.input, outputs=output)

In [6]:
New_resnet_model.compile(optimizer=Adam(lr=1e-4),
                         loss='categorical_crossentropy',
                         metrics=['accuracy'])

In [7]:
print(New_resnet_model.summary())

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [8]:
New_resnet_model.fit(x_train, y_train, 
                     batch_size=batch_size, 
                     epochs=epochs,
                     validation_data=(x_test,y_test))

Epoch 1/30
782/782 [==============================] - 26s 34ms/step - loss: 1.6693 - accuracy: 0.4886 - val_loss: 2.0953 - val_accuracy: 0.3353
Epoch 2/30
782/782 [==============================] - 26s 33ms/step - loss: 0.8594 - accuracy: 0.7216 - val_loss: 0.7504 - val_accuracy: 0.7479
Epoch 3/30
782/782 [==============================] - 26s 33ms/step - loss: 0.6161 - accuracy: 0.7956 - val_loss: 0.6307 - val_accuracy: 0.7900
Epoch 4/30
782/782 [==============================] - 25s 32ms/step - loss: 0.4809 - accuracy: 0.8424 - val_loss: 0.6172 - val_accuracy: 0.7969
Epoch 5/30
782/782 [==============================] - 25s 33ms/step - loss: 0.3569 - accuracy: 0.8803 - val_loss: 0.6255 - val_accuracy: 0.8039
Epoch 6/30
782/782 [==============================] - 25s 32ms/step - loss: 0.3097 - accuracy: 0.8996 - val_loss: 0.7581 - val_accuracy: 0.7824
Epoch 7/30
782/782 [==============================] - 27s 34ms/step - loss: 0.2445 - accuracy: 0.9204 - val_loss: 0.6988 - val_accuracy:

In [9]:
score = New_resnet_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.914863646030426
Test accuracy: 0.8241999745368958
